In [1]:
import pandas as pd
import numpy as np
from wikidata.QueryManager import *

# Data extraction : sport quotes & occupation of the speakers

In [ ]:
# Load dataset containing sport quotes
quotes_directory = 'out_bz2'
sport_quotes_2015 = pd.read_csv(quotes_directory + '/sport-quotes-2015.csv.bz2', compression='bz2', usecols=['qids'])
sport_quotes_2016 = pd.read_csv(quotes_directory + '/sport-quotes-2016.csv.bz2', compression='bz2', usecols=['qids'])
sport_quotes_2017 = pd.read_csv(quotes_directory + '/sport-quotes-2017.csv.bz2', compression='bz2', usecols=['qids'])
sport_quotes_2018 = pd.read_csv(quotes_directory + '/sport-quotes-2018.csv.bz2', compression='bz2', usecols=['qids'])
sport_quotes_2019 = pd.read_csv(quotes_directory + '/sport-quotes-2019.csv.bz2', compression='bz2', usecols=['qids'])
sport_quotes_2020 = pd.read_csv(quotes_directory + '/sport-quotes-2020.csv.bz2', compression='bz2', usecols=['qids'])

df_all = pd.concat([sport_quotes_2015, sport_quotes_2016, sport_quotes_2017, sport_quotes_2018, sport_quotes_2019, sport_quotes_2020])
df_all.reset_index(inplace=True, drop=True)
df_all.head()

# Load speaker attributes from Google Drive
speaker_attributes = pd.read_parquet('speaker_attributes.parquet', columns=['party', 'occupation', 'id'])

# Load the database relating qids with wikidata labels
qid_labels = pd.read_csv('wikidata_labels_descriptions_quotebank.csv.bz2', compression='bz2', index_col='QID')


In [ ]:
# Removing square brackets and quotation marks from qids to match the speaker_attributes dataset
df_all['qids'] = df_all['qids'].str.strip('[\'\']')
speakers = speaker_attributes[speaker_attributes['id'].isin(df_all['qids'])]
speakers 



# Data extraction : politicians sport quotes & party of politicians

In [2]:
#loading quotes dataset

#for in range(2015, 2021):
#    df
df_2015 = pd.read_csv('quotes_by_politicians_2015.csv')
df_2016 = pd.read_csv('quotes_by_politicians_2016.csv')
df_2017 = pd.read_csv('quotes_by_politicians_2017.csv')
df_2018 = pd.read_csv('quotes_by_politicians_2018.csv')
df_2019 = pd.read_csv('quotes_by_politicians_2019.csv')
df_2020 = pd.read_csv('quotes_by_politicians_2020.csv')

df = pd.concat([df_2015, df_2016, df_2017, df_2018, df_2019, df_2020])
df.reset_index(inplace=True, drop=True)
df.head()

,index_q,quoteID,quotation,speaker,qids,date
0,0,2015-08-03-026031,I am not a poster child for the Christie admin...,Bill Pascrell,Q529090,2015-08-03 17:44:16
1,1,2015-08-03-002773,Accidents occur where the street meets the rai...,Bill Pascrell,Q529090,2015-08-03 17:44:16
2,2,2015-02-26-089365,These federal investments will go a long way t...,Bill Pascrell,Q529090,2015-02-26 07:55:10
3,3,2015-02-14-027291,"Looking back, it seems that every several mont...",Bill Pascrell,Q529090,2015-02-14 00:12:20
4,4,2015-08-03-088419,This is just one of many crossings that need i...,Bill Pascrell,Q529090,2015-08-03 17:44:16


In [3]:
for df_ann in [df_2015, df_2016, df_2017, df_2018, df_2019, df_2020]:
    print(df_ann.shape)

df.shape

(65212, 6)
(48014, 6)
(99647, 6)
(84907, 6)
(52910, 6)
(16470, 6)


(367160, 6)

In [4]:
df['current_party'] = [None] * df.shape[0]
#df.loc[2, 'current_party'] = "salut"
df.head()

,index_q,quoteID,quotation,speaker,qids,date,current_party
0,0,2015-08-03-026031,I am not a poster child for the Christie admin...,Bill Pascrell,Q529090,2015-08-03 17:44:16,None
1,1,2015-08-03-002773,Accidents occur where the street meets the rai...,Bill Pascrell,Q529090,2015-08-03 17:44:16,None
2,2,2015-02-26-089365,These federal investments will go a long way t...,Bill Pascrell,Q529090,2015-02-26 07:55:10,None
3,3,2015-02-14-027291,"Looking back, it seems that every several mont...",Bill Pascrell,Q529090,2015-02-14 00:12:20,None
4,4,2015-08-03-088419,This is just one of many crossings that need i...,Bill Pascrell,Q529090,2015-08-03 17:44:16,None


In [3]:
manager = QueryManager()

# for df_ann in [df_2015, df_2016, df_2017, df_2018, df_2019, df_2020]:
for df_ann in [df_2019]:
    df_ann['current_party'] = [None] * df_ann.shape[0]
    
    for i in range(df_ann.shape[0]):
        date = str(df_ann['quoteID'].loc[i][0:7])
        
        try:
            politician_qid = df_ann['qids'].loc[i]
            party_qid = manager.search_politician_party(politician_qid, date)
            df_ann.loc[i, 'current_party'] = party_qid
        except Exception as e:
            print(f"Error encountered for qid '{df_ann['qids'].loc[i]}': ", e)
        
        if i % 512 == 0:
            print(f'finished another chunks: {i}/{df_ann.shape[0]}')
            manager.stats_TODO_remove()
    
    
    year = df_ann['quoteID'].loc[i][0:4]
    df_ann.to_csv('quotes_by_politician_with_party/' + year + '.csv')
    
    print(f'>>> finished year {year}')

finished another chunks: 0/52910
  - gotten from online: 1 vs gotten from cache 0
finished another chunks: 512/52910
  - gotten from online: 1 vs gotten from cache 512
finished another chunks: 1024/52910
  - gotten from online: 1 vs gotten from cache 1024
finished another chunks: 1536/52910
  - gotten from online: 6 vs gotten from cache 1531
finished another chunks: 2048/52910
  - gotten from online: 6 vs gotten from cache 2043
finished another chunks: 2560/52910
  - gotten from online: 6 vs gotten from cache 2555
finished another chunks: 3072/52910
  - gotten from online: 8 vs gotten from cache 3065
finished another chunks: 3584/52910
  - gotten from online: 11 vs gotten from cache 3574
finished another chunks: 4096/52910
  - gotten from online: 11 vs gotten from cache 4086
finished another chunks: 4608/52910
  - gotten from online: 12 vs gotten from cache 4597
finished another chunks: 5120/52910
  - gotten from online: 17 vs gotten from cache 5104
finished another chunks: 5632/52910


In [ ]:
# Load speaker attributes from Google Drive
speaker_attributes = pd.read_parquet('speaker_attributes.parquet', columns=['occupation','party','id'])

# Load the database relating qids with wikidata labels
qid_labels = pd.read_csv('wikidata_labels_descriptions_quotebank.csv.bz2', compression='bz2', index_col='QID') 

In [ ]:
# Removing square brackets and quotation marks from qids to match the speaker_attributes dataset
df['qids'] = df['qids'].str.strip('[\'\']')
politicians = speaker_attributes[speaker_attributes['id'].isin(df['qids'])]

politicians

# Which political parties or politicians are the most present in the sports quotes?

## Parties

In [ ]:
# Only keeping the first qid in the party feature
politicians_1_qid = politicians['current_party']
politicians_1_qid.shape

#OLD TO DELETE
#politicians_1_qid = politicians['party'].str.get(0)
#politicians_1_qid.shape

# Counts the occurences of each party and merging the corresponding label
party_counts_1_qid = pd.DataFrame(pd.value_counts(politicians_1_qid))
party_counts_label_1_qid = party_counts_1_qid.merge(qid_labels, left_index=True, right_on='QID').rename(columns={'party': 'count'})
party_counts_label_1_qid.head(3)

In [ ]:
party_counts_label_1_qid.to_csv('party_count.csv', index=False)

In [ ]:
# Setting label of party with count lower than the .99 quantile to 'Others'
threshold_value = party_counts_label_1_qid['count'].quantile(.99)
values_below_thresh = party_counts_label_1_qid['count'] < threshold_value
party_counts_label_1_qid.loc[values_below_thresh,'Label'] = 'Others'

In [ ]:
party_counts_label_1_qid.groupby('Label').sum().plot(kind='pie', subplots=True, figsize=(10, 10), legend=False, autopct='%1.f%%', title='Parties distribution over sport quotes')

In [ ]:
party_counts_label_1_qid.groupby('Label').sum().sort_values(by='count', ascending=False).plot(kind='bar',legend=False, figsize=(8, 8), log=False, title='Parties distribution over sport quotes')

## Politicians

In [ ]:
# Only keeping the first qid in the party feature
politicians_1_qid = politicians['id']
politicians_1_qid

# Counts the occurences of each party and merging the corresponding label
politician_counts_1_qid = pd.DataFrame(pd.value_counts(politicians_1_qid))
politician_counts_label_1_qid = politician_counts_1_qid.merge(qid_labels, left_index=True, right_on='QID').rename(columns={'politician': 'count'})
politician_counts_label_1_qid.head(3)


In [ ]:
# Setting label of politician with count lower than the .99 quantile to 'Others'
threshold_value = politician_counts_label_1_qid['count'].quantile(.99)
values_below_thresh = politician_counts_label_1_qid['count'] < threshold_value
politician_counts_label_1_qid.loc[values_below_thresh,'Label'] = 'Others'

In [ ]:
politician_counts_label_1_qid.groupby('Label').sum().plot(kind='pie', subplots=True, figsize=(10, 10), legend=False, autopct='%1.f%%', title='Politicians distribution over sport quotes')

# All sport quotes speakers : occupation distribution

In [ ]:
occupations = []

for i in range(speakers.shape[0]):
    occupations = np.concatenate((occupations, speakers['occupation'].iloc[i]), axis=None)

occupations = pd.DataFrame(occupations, columns=['occupation'])

In [ ]:
# Counts the occurences of each occupation and merging the corresponding label
occupation_counts_1_qid_speakers = pd.DataFrame(pd.value_counts(occupations['occupation']))
occupation_counts_label_1_qid_speakers = occupation_counts_1_qid_speakers.merge(qid_labels, left_index=True, right_on='QID').rename(columns={'occupation': 'count'})
occupation_counts_label_1_qid_speakers

In [ ]:
# Setting label of occupation with count lower than the .99 quantile to 'Others'
threshold_value = occupation_counts_label_1_qid_speakers['count'].quantile(.99)
values_below_thresh = occupation_counts_label_1_qid_speakers['count'] < threshold_value
occupation_counts_label_1_qid_speakers.loc[values_below_thresh,'Label'] = 'Others'

In [ ]:
# plot
occupation_counts_label_1_qid_speakers.groupby('Label').sum().plot(kind='pie', subplots=True, figsize=(10, 10), legend=False, autopct='%1.f%%', title='All sport quotes speakers occupations')

In [ ]:
# Counts the occurences of each occupation and merging the corresponding label
occupation_counts_1_qid_speakers = pd.DataFrame(pd.value_counts(occupations['occupation']))
occupation_counts_label_1_qid_speakers = occupation_counts_1_qid_speakers.merge(qid_labels, left_index=True, right_on='QID').rename(columns={'occupation': 'count'})
occupation_counts_label_1_qid_speakers

# Setting label of occupation with political party
values_below_thresh = ~(speakers['party'].isna())
occupation_counts_label_1_qid_speakers.loc[values_below_thresh,'Label'] = 'With political party'

# Setting label of occupation with count lower than the .99 quantile to 'Others'
threshold_value = occupation_counts_label_1_qid_speakers['count'].quantile(.99)
values_below_thresh = occupation_counts_label_1_qid_speakers['count'] < threshold_value
occupation_counts_label_1_qid_speakers.loc[values_below_thresh,'Label'] = 'Others'

In [ ]:
# plot
occupation_counts_label_1_qid_speakers.groupby('Label').sum().plot(kind='pie', subplots=True, figsize=(10, 10), legend=False, autopct='%1.f%%', title='All sport quotes speakers occupations (grouping people with a political party)')

# Politicians : occupation distribution

In [ ]:
politicians['occupation']

In [ ]:
politiciansbis=politicians

In [ ]:
politicians_none_qid = []
politicians_1_qid = []
politicians_many_qid = []

for i in range(politiciansbis.shape[0]):
    if politiciansbis['occupation'].iloc[i] is None:
        politicians_none_qid.append(politiciansbis['occupation'].iloc[i])
    elif len(politiciansbis['occupation'].iloc[i]) == 1 :
           politicians_1_qid.append(politiciansbis['occupation'].iloc[i])
    elif len(politiciansbis['occupation'].iloc[i]) > 1 :
             politicians_many_qid.append(politiciansbis['occupation'].iloc[i])
    else :
             politicians_none_qid.append(politiciansbis['occupation'].iloc[i])

politicians_none_qid = pd.DataFrame(politicians_none_qid, columns=['occupation'])
politicians_1_qid = pd.DataFrame(politicians_1_qid, columns=['occupation'])
#politicians_1_qid

In [ ]:
politicians_many_qid_politician = np.array([])
politicians_many_qid_no_politician = np.array([])
for item in politicians_many_qid :
    if 'Q82955' in item :
        politicians_many_qid_politician = np.concatenate((politicians_many_qid_politician, item), axis=None)
    else : 
        politicians_many_qid_no_politician = np.concatenate((politicians_many_qid_no_politician, item), axis=None)

politicians_many_qid_politician = pd.DataFrame(politicians_many_qid_politician, columns=['occupation'])
politicians_many_qid_no_politician = pd.DataFrame(politicians_many_qid_no_politician, columns=['occupation'])
#politicians_many_qid_politician

In [ ]:
politicians_none_qid

In [ ]:
# Counts the occurences of each occupation and merging the corresponding label FOR 1 OCCUPATION
occupation_counts_1_qid = pd.DataFrame(pd.value_counts(politicians_1_qid['occupation']))
occupation_counts_label_1_qid = occupation_counts_1_qid.merge(qid_labels, left_index=True, right_on='QID').rename(columns={'occupation': 'count'})
occupation_counts_label_1_qid.head()
occupation_counts_label_1_qid.tail()

# Counts the occurences of each occupation and merging the corresponding label FOR MANY OCCUPATIONS - POLITICIAN
occupation_counts_many_qid_politician = pd.DataFrame(pd.value_counts(politicians_many_qid_politician['occupation']))
occupation_counts_label_many_qid_politician = occupation_counts_many_qid_politician.merge(qid_labels, left_index=True, right_on='QID').rename(columns={'occupation': 'count'})
occupation_counts_label_many_qid_politician.head()
occupation_counts_label_many_qid_politician.tail()

# Counts the occurences of each occupation and merging the corresponding label FOR MANY OCCUPATIONS - NO POLITICIAN
occupation_counts_many_qid_no_politician = pd.DataFrame(pd.value_counts(politicians_many_qid_no_politician['occupation']))
occupation_counts_label_many_qid_no_politician = occupation_counts_many_qid_no_politician.merge(qid_labels, left_index=True, right_on='QID').rename(columns={'occupation': 'count'})
occupation_counts_label_many_qid_no_politician

In [ ]:
# Setting label of occupation with count lower than the .98 quantile to 'Others'
threshold_value = occupation_counts_label_1_qid['count'].quantile(.98)
values_below_thresh = occupation_counts_label_1_qid['count'] < threshold_value
print(occupation_counts_label_1_qid.loc[values_below_thresh,'Label'])
occupation_counts_label_1_qid.loc[values_below_thresh,'Label'] = 'Others'

# Delete politician label because not relevant for the study
occupation_counts_label_many_qid_politician = occupation_counts_label_many_qid_politician[occupation_counts_label_many_qid_politician['Label']!='politician']

# Setting label of occupation with count lower than the .98 quantile to 'Others'
threshold_value = occupation_counts_label_many_qid_politician['count'].quantile(.98)
values_below_thresh = occupation_counts_label_many_qid_politician['count'] < threshold_value
print(occupation_counts_label_many_qid_politician.loc[values_below_thresh,'Label'])
occupation_counts_label_many_qid_politician.loc[values_below_thresh,'Label'] = 'Others'

# Setting label of occupation with count lower than the .98 quantile to 'Others'
threshold_value = occupation_counts_label_many_qid_no_politician['count'].quantile(.98)
values_below_thresh = occupation_counts_label_many_qid_no_politician['count'] < threshold_value
print(occupation_counts_label_many_qid_no_politician.loc[values_below_thresh,'Label'])
occupation_counts_label_many_qid_no_politician.loc[values_below_thresh,'Label'] = 'Others'

In [ ]:
# plot
occupation_counts_label_1_qid.groupby('Label').sum().plot(kind='pie', subplots=True, figsize=(10, 10), legend=False, autopct='%1.1f%%', title='Sport quotes speakers with politicial party with single occupation')

In [ ]:
# plot
occupation_counts_label_many_qid_politician.groupby('Label').sum().plot(kind='pie', subplots=True, figsize=(10, 10), legend=False, autopct='%1.f%%', title='Sport quotes speakers with politicial party with multiple occupations including politician')

In [ ]:
# plot
occupation_counts_label_many_qid_no_politician.groupby('Label').sum().plot(kind='pie', subplots=True, figsize=(10, 10), legend=False, autopct='%1.f%%', title='Sport quotes speakers with politicial party with multiple occupations excluding politician')

# Notes

In [ ]:

#au moins politician comparé à ceux qui reste

#-Would have also been interesting to visualise (as a type negative control) in a more exploratory sense how the # of politicians in sports-related news differs from other speakers that are unrelated to politics. This would be useful to show that trends for the politicians in sports do not follow some baseline trend.


In [ ]:
#Removing speakers without any known party (i.e. with 'party' == None)
#politicians = speakers[~(speakers['party'].isna())]

#politician_qid= qid_labels[qid_labels['Label']=='politician']
#politicians = speakers[politician_qid.index.isin(speakers['occupation'])]